<a href="https://colab.research.google.com/github/zjzsu2000/CMPE258/blob/master/assignment4/b)_2_VGG16_cifar_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Graded Assignment 4 : CNNs
b) CNN basics : use pytorch and from scratch (not using existing architecture shipped models)  write basic LeNet (mnist) and VGG-16 model (cifar100) and and RESNET (only 8 layers - not 52)  architectures - run on gpu  (use .cuda instruction) - train and report all the stats

 # b) 2_VGG16-cifar_pytorch

In [0]:
import torch
import torch.nn as nn
from torch.nn import Module,Linear




In [2]:
VGG16_layers=[64, 64, 'Max', 128, 128,'Max', 256, 256, 256, 256, 'Max', 512, 512, 512, 512, 'Max', 512, 512, 512, 512, 'Max']
class VGG_16(Module):    
    def __init__(self, VGG16):
        super(VGG_16, self).__init__()
        self.features = self._make_layers(VGG16_layers)
        self.classifier = Linear(512, 10)

    def forward(self, x):
        out = self.features(x)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        return out

    def _make_layers(self,VGG16_layers):
        layers = []
        in_channels = 3
        for x in VGG16_layers:
            if x != 'Max':
                layers += [nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
                           nn.BatchNorm2d(x),
                           nn.ReLU(inplace=True)]
                in_channels = x
                
            else:
              layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
                
        layers += [nn.AvgPool2d(kernel_size=1, stride=1)]
        return nn.Sequential(*layers)


#test
x = torch.randn(2,3,32,32)
y = VGG_16(x)
y

VGG_16(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 

# Train CIFAR10 with PyTorch.


In [0]:
import torch
import torch.nn as nn
import os,argparse
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
from torchvision import transforms



Using GPU

In [3]:
torch.manual_seed(42)
use_cuda = torch.cuda.is_available()
print(use_cuda)

device = torch.device("cuda" if use_cuda else "cpu")

device

True


device(type='cuda')

## Load data

In [6]:
transform_train = transforms.Compose([transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)


Extracting ./data/cifar-10-python.tar.gz to ./data


In [7]:
transform_test = transforms.Compose([transforms.ToTensor(),
                      transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),])

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)


Files already downloaded and verified


In [0]:
classes = ('plane', 'car', 'bird', 'cat', 'deer','dog', 'frog', 'horse', 'ship', 'truck')

In [10]:
lr = 0.01

best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# Model
print('==> Building model..')
net = VGG_16(VGG16_layers).to(device)


if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

==> Building model..


In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr,momentum=0.9, weight_decay=5e-4)


In [0]:
# Training
def training(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        print(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                     % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))


In [0]:
def testing(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            print(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                         % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt.pth')
        best_acc = acc




In [18]:
for epoch in range(0, 30):
    training(epoch)
    testing(epoch)

Streaming output truncated to the last 5000 lines.
34 100 Loss: 0.449 | Acc: 86.200% (3017/3500)
35 100 Loss: 0.453 | Acc: 86.139% (3101/3600)
36 100 Loss: 0.462 | Acc: 85.973% (3181/3700)
37 100 Loss: 0.462 | Acc: 86.026% (3269/3800)
38 100 Loss: 0.461 | Acc: 86.128% (3359/3900)
39 100 Loss: 0.461 | Acc: 86.125% (3445/4000)
40 100 Loss: 0.460 | Acc: 86.122% (3531/4100)
41 100 Loss: 0.457 | Acc: 86.238% (3622/4200)
42 100 Loss: 0.452 | Acc: 86.349% (3713/4300)
43 100 Loss: 0.452 | Acc: 86.409% (3802/4400)
44 100 Loss: 0.451 | Acc: 86.467% (3891/4500)
45 100 Loss: 0.451 | Acc: 86.478% (3978/4600)
46 100 Loss: 0.451 | Acc: 86.489% (4065/4700)
47 100 Loss: 0.456 | Acc: 86.354% (4145/4800)
48 100 Loss: 0.454 | Acc: 86.449% (4236/4900)
49 100 Loss: 0.457 | Acc: 86.420% (4321/5000)
50 100 Loss: 0.457 | Acc: 86.431% (4408/5100)
51 100 Loss: 0.457 | Acc: 86.462% (4496/5200)
52 100 Loss: 0.459 | Acc: 86.396% (4579/5300)
53 100 Loss: 0.457 | Acc: 86.444% (4668/5400)
54 100 Loss: 0.456 | Acc: 86.

Acc: 89.240%

In [19]:
os.path.isdir('checkpoint')

True